# Spatial changes detection in spectral indices values (in wetlands)

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [1]:
import ee
import geemap,eemont

import sys
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

import sys

sys.path.append('../preprocessing/clouds')
import landsat_preprocessing
import s2_preprocessing

sys.path.append('../wetlands_detection')
import wetlands_unsupervised_clustering

import changes_btw_2_dates

In [2]:
roi = ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])
s2_1 = s2_preprocessing.get_s2_cloud_free_col_dates(roi, 0.3,'2017-05-01', '2017-10-01', ['NDVI', 'MNDWI','NDMI']).median()
s2_2 = s2_preprocessing.get_s2_cloud_free_col_dates(roi, 0.3,'2023-05-01', '2023-10-01', ['NDVI', 'MNDWI','NDMI']).median()

l8_1 = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'2017-05-01', '2017-10-01').median()
l8_2 = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'2023-05-01', '2023-10-01').median()

l7_1 = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'1999-05-01', '1999-10-01').median()
l7_2 = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'2023-05-01', '2023-10-01').median()

In [18]:
import geemap.chart as chart

samples = s2_2.sample(
        numPixels=10000,
        #classBand='label', 
        scale=10, 
        region = roi
    )

property = 'NDVI'

options = {
    "title": "Histogramm of NDVI band summer 2023",
    "xlabel": "NDVI band value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)

In [13]:
import sys
sys.path.append('../../wetlands_detection')
import wetlands_unsupervised_clustering

min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')
wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)

In [ ]:
def diff_image(roi, band, image_1, image_2):

    band_img_1 = image_1.select(band)
    band_img_2 = image_2.select(band)

    diff = band_img_2.subtract(band_img_1).rename('diff')

    max = ee.Number(diff.reduceRegion(reducer = ee.Reducer.stdDev(), geometry = roi, scale = 10).get('diff'))
    min = max.multiply(-1)

    return diff

In [100]:
thresholdGain = ee.Number(s2_1.reduceRegion(reducer = ee.Reducer.stdDev(), geometry = roi, scale = 10).get('NDMI'))
display(thresholdGain)

In [101]:
diffs2_ndvi, diffClassifieds2_ndvi = changes_btw_2_dates.getChanges(roi, 0.4, 'NDMI', s2_1, s2_2)

In [102]:

s2_rgbVis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 0.1}
# Créer une palette pour les classes
palette = ['00FF00', 'FF0000']  # Vert pour gain, rouge pour perte

# Masquer les zones sans changement (classe 0) et appliquer le masque des zones humides
diffClassifiedFiltered = diffClassifieds2_ndvi.updateMask(diffClassifieds2_ndvi.neq(0))#.updateMask(wetlands)

# Visualiser l'image classifiée avec la palette
diffClassifiedVis = diffClassifiedFiltered.visualize(min=1, max=2, palette=palette)

m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(s2_2.divide(10000), s2_rgbVis, 's2_img')
m.addLayer(diffClassifiedVis, {}, 'Change Detection')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [23]:
import geemap.chart as chart

samples = l8_2.sample(
        numPixels=10000,
        #classBand='label', 
        scale=10, 
        region = roi
    )

property = 'NDVI'

options = {
    "title": "Histogramm of NDVI band summer 2023",
    "xlabel": "NDVI band value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)

In [96]:
thresholdGainl8 = ee.Number(l8_1.reduceRegion(reducer = ee.Reducer.stdDev(), geometry = roi, scale = 10).get('NDMI'))
display(thresholdGainl8)

In [97]:
diffl8_ndvi, diffClassifiedl8_ndvi = changes_btw_2_dates.getChanges(roi, 0.08, 'NDMI', l8_1, l8_2)

In [98]:
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

l8_rgbVis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.1}

# Créer une palette pour les classes
palette = ['00FF00', 'FF0000']  # Vert pour gain, rouge pour perte
diffClassifiedFiltered = diffClassifiedl8_ndvi.updateMask(diffClassifiedl8_ndvi.neq(0))#.updateMask(wetlands)
# Visualiser l'image classifiée
diffClassifiedVis = diffClassifiedFiltered.visualize(min=1, max=2, palette=palette)
m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(apply_scale_factors(l8_1), l8_rgbVis, 'l8_img')
m.addLayer(diffClassifiedVis, {}, 'Change Detection')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
#m.addLayer(l8_1.select('NDVI'),{}, 'ndvi')
m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')

m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [108]:
l7_1 = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'1999-05-01', '1999-10-30').median()
l7_2 = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI','NDMI'],0.3,'2023-05-01', '2023-10-30').median()

thresholdGainl7 = ee.Number(l7_1.reduceRegion(reducer = ee.Reducer.stdDev(), geometry = roi, scale = 30).get('NDMI'))
display(thresholdGainl7)

In [109]:
thresholdGainl7_1 = ee.Number(l7_1.updateMask(wetlands).reduceRegion(reducer = ee.Reducer.median(), geometry = roi, scale = 30).get('NDMI'))
thresholdGainl7_2 = ee.Number(l7_2.updateMask(wetlands).reduceRegion(reducer = ee.Reducer.median(), geometry = roi, scale = 30).get('NDMI'))

display(thresholdGainl7_1,thresholdGainl7_2)

In [110]:
diffl7_ndvi, diffClassifiedl7_ndvi = changes_btw_2_dates.getChanges(roi, 0.07, 'NDVI', l7_1, l7_2)

In [111]:
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

l7_rgbVis = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.1}

# Créer une palette pour les classes
palette = ['00FF00', 'FF0000']  # Vert pour gain, rouge pour perte

diffClassifiedFiltered = diffClassifiedl7_ndvi.updateMask(diffClassifiedl7_ndvi.neq(0)).selfMask() #updateMask(wetlands)
# Visualiser l'image classifiée
diffClassifiedVis = diffClassifiedFiltered.visualize(min=1, max=2, palette=palette) #updateMask(wetlands)
m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(apply_scale_factors(l7_1), l7_rgbVis, 'l8_img')
m.addLayer(diffClassifiedVis, {}, 'Change Detection')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…